## Import Data and Libraries

In [1]:
!pip install catboost
!pip install lightgbm
!pip install missingpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 54.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 185.7 MB/s eta 0:00:00


## Load Data and Libraries

In [3]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, f1_score
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [4]:
dataset = pd.read_csv("Dataset/heg_sample_data.csv") #normalized dataset
dataset.drop('Unnamed: 0', axis=1, inplace=True)
dataset.shape, dataset.columns

((13652, 72),
 Index(['days_from_entrance', 'age', 'document.sexo', 'UTI',
        'delta_collect_timestamp_t-t1', 'delta_collect_timestamp_t1-t2',
        'delta_collect_timestamp_t2-t3', 'delta_collect_timestamp_t3-t4',
        'document.freq_cardiaca(t)', 'document.freq_cardiaca(t-1)',
        'document.freq_cardiaca(t-2)', 'document.freq_cardiaca(t-3)',
        'document.freq_cardiaca(t-4)', 'document.freq_respiratoria(t)',
        'document.freq_respiratoria(t-1)', 'document.freq_respiratoria(t-2)',
        'document.freq_respiratoria(t-3)', 'document.freq_respiratoria(t-4)',
        'document.glicemia_capilar(t)', 'document.glicemia_capilar(t-1)',
        'document.glicemia_capilar(t-2)', 'document.glicemia_capilar(t-3)',
        'document.glicemia_capilar(t-4)', 'document.pa_diastolica(t)',
        'document.pa_diastolica(t-1)', 'document.pa_diastolica(t-2)',
        'document.pa_diastolica(t-3)', 'document.pa_diastolica(t-4)',
        'document.pa_sistolica(t)', 'document.pa_si

## Setup Expetiments

In [5]:
X = dataset.drop(["outcome"], axis = 1)
Y = dataset["outcome"]

In [6]:
kfold = KFold(n_splits=10, random_state=7, shuffle=True)

classifiers = {
    'XGBoost' : XGBClassifier(learning_rate=0.1, n_estimators=100,random_state=7, tree_method='gpu_hist'),
    'LogReg': LogisticRegression(solver='liblinear', multi_class='ovr'),
    'D.Tree': DecisionTreeClassifier(),
    'RForest': RandomForestClassifier(n_estimators = 50),
    'CatBoos': CatBoostClassifier(learning_rate=0.1,n_estimators=100,random_state=7,task_type='GPU',verbose = False),
    'Naive': GaussianNB(),
    'Light': lgb.LGBMClassifier()
}

## Run Basic Experiments

In [7]:
for c in classifiers:
  start = time.time()
  model = classifiers[c]
  scores = cross_val_score(model, X, Y, cv=kfold, scoring='roc_auc')
  scores_f1 = cross_val_score(model, X, Y, cv=kfold, scoring='f1')
  print (c + '\t', round(scores.mean(),4), '(' + str(round(scores_f1.mean(),4)) + ')', round(time.time() - start,2), 's')

[10:28:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:28:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:28:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:28:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

## Cross Validation by Windowing

In [8]:
cols = ['age', 'document.sexo', 'UTI', 'days_from_entrance']
t_cols = [c for c in dataset.columns if '4)' in c and (not 'time' in c)]

for i in [4,3,2,1,0]:
  
  if i == 4: cols.extend(t_cols)
  if i == 0:
    tN_cols = [c for c in dataset.columns if ('t)' in c or '_t-' in c) and (not 'time' in c)]
    cols.extend(tN_cols)
  else: 
    tN_cols = [c for c in dataset.columns if ('t-'+str(i) in c or '_t'+str(i) in c) and (not 'time' in c)]
    cols.extend(tN_cols)

  cols = list(set(cols))
  print('Number of Columns:', len(cols), 'Exam(s):', 5-i)
  print(cols)

  X_W = dataset[cols]
  Y_W = dataset["outcome"]

  for c in classifiers:
    start = time.time()
    model = classifiers[c]
    scores = cross_val_score(model, X_W, Y_W, cv=kfold, scoring='roc_auc')
    print ('\t' + c + '\t', round(scores.mean(),4), '(+-' + str(round(scores.std(),4)) + ')', round(time.time() - start,2), 's')

Number of Columns: 11 Exam(s): 1
['document.sexo', 'document.pa_diastolica(t-4)', 'UTI', 'document.pa_sistolica(t-4)', 'document.freq_respiratoria(t-4)', 'document.glicemia_capilar(t-4)', 'age', 'document.temperatura(t-4)', 'document.sat_o2(t-4)', 'days_from_entrance', 'document.freq_cardiaca(t-4)']
[10:31:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:31:24] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:31:24] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly se